In [ ]:
%pylab inline
plt.style.use('bmh')

import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd

# Intro

Pandas dataframes are great for data exploration. But they have some limitations. One of them is that it's not possible to use geospatial information in exploratory data analysis. GeoPandas aims at solving this problem.

GeoPandas provides two abstractions: `GeoDataFrame` and `GeoSeries`. At their core, they are usual dataframes and series, but with each row being equipped with geometry. This allows to perform more complex queries, for example:

- select all the rows, which are inside some area,
- select all the rows with underlying geometry area above threshold.

GeoPandas also provides a lot of typical geospatiial operations in a vectorized form similar to `apply`, `groupby` and `join`, as well as I/O operations like saving to PostGIS, or reading GeoJSON or shapefiles.

# Shapelly geometries

The main Python package, which provides geometry operations, is **Shapely**. It contains all the main geometry primitives (points, lines, polygons, multi-ppolygons, etc.) and common operations on them.

Shapely also knows how to visualize the geometry, which is very useful during EDA. Note, however, that Shapely knows nothing about coordinate reference systems, and this functionality is provided by other packages, which GeoPandas relies on.

In [ ]:
import shapely

The most simple primitive geometry type is point:

In [ ]:
point = shapely.geometry.Point(0.25, 0.5)
point

Naturally, point has no area:

In [ ]:
point.area

Polygons are created from list of points or pairs of coordinates:

In [ ]:
polygon = shapely.geometry.Polygon([(0, 0), (0, 1), (1, 1), (1, 0)])
polygon

In [ ]:
polygon = shapely.geometry.Polygon([(0, 0), (0, 1), (1, 1)])
polygon

We can easily calculate polygon area, or it's bounding box:

In [ ]:
polygon.area, polygon.bounds

Polygon boundary may be extracted as well:

In [ ]:
polygon.boundary

Note, that, again, point boundary is just empty:

In [ ]:
point.boundary

Let's check if polygon boundary is a polygon itself (it's not, of course):

In [ ]:
type(polygon.boundary), polygon.bounds

We can create lines strings directly:

In [ ]:
line = shapely.geometry.LineString([(0, 0), (0, 1), (1, 1)])
line

Order of points matters for line strings:

In [ ]:
line = shapely.geometry.LineString([(0, 0), (1, 1), (0, 1)])
line

Is it for polygon?

In [ ]:
polygon = shapely.geometry.Polygon([(0, 0), (1, 1), (0, 1)])
polygon

Shapely provides binary geometric operations:

In [ ]:
point.within(polygon)

In [ ]:
polygon_alt = shapely.geometry.Polygon([(0, 1), (1, 1), (1, 0)])
polygon_alt

In [ ]:
polygon.intersection(polygon_alt)

In [ ]:
polygon.union(polygon_alt)

We can create simpler primitives directly (boxes are very common and there's a shortcut in Shapely):

In [ ]:
bbox = shapely.geometry.box(0.25, 0.25, 0.75, 0.75)
bbox

In [ ]:
polygon.union(bbox)

In [ ]:
polygon.difference(bbox)

In [ ]:
polygon.symmetric_difference(bbox)

# Reading shapefiles

There are various formats to store geospatial data. One of the most common is shapefile. It's a binary format and is typically read with Fiona package or directly with GeoPandas.

To experiment, we will download shapefile of London area from [London Datastore](https://data.london.gov.uk/dataset/statistical-gis-boundary-files-london).

Specifically, we will use `London_Ward_CityMerged.shp`, extracted from `ESRI` directory in `statistical-gis-boundaries-london.zip`. Note, that you also need `.shx` file (it contains offset and other information for proper reading of shapefile). You can take these files directly from the campus, but we encourage you to play with other files on that page as well.

The main entry point for reading files with geospatial information is `gpd.read_file`:

In [ ]:
london_areas = gpd.read_file("London_Ward_CityMerged.shp")

Similar to usual dataframes, GeoPandas provides plotting directly on geo-dataframes. In this case we plot polygons and color them by their integer index (you can reverse-engineer the groupping of wards from the plot): 

In [ ]:
london_areas.plot(column=london_areas.index.values)

In [ ]:
print(f"Total area, km2: {london_areas.area.sum()/1e6}")

# Creating `GeoDataFrame` from a usual dataframe

Now that we know how to read some shapefiles and can use them to filter our data, we need to create an actual `GeoDataFrame`. For this, we'll use assidents data from time series notebook. Note, that both London wards and accidents use easting and northing and not global `lat/lon` coordinates. This is typical when working with country level data, as local coordinates are linear and in meters and are much faster and convenient to work with.

In [ ]:
df = pd.read_csv('accidents_2005_to_2007.csv.zip',
                 usecols=["Accident_Index", "Accident_Severity", "Number_of_Vehicles",
                          "Number_of_Casualties", "Year",
                          "Location_Easting_OSGR", "Location_Northing_OSGR"])

We now create a `GeoDataFrame` by specifying the geometry directly:

In [ ]:
gdf = gpd.GeoDataFrame(df[["Accident_Index", "Year", "Accident_Severity", "Number_of_Vehicles", "Number_of_Casualties"]],
                       geometry=gpd.points_from_xy(*df[["Location_Easting_OSGR", "Location_Northing_OSGR"]].values.T))
gdf

Coordinates are provided in separate columns, hence, we have to create geomerties manually. In general, GeoPandas knows how to handle `geometry` column in many formats, most common being WKT (Well Known Text).

We can now easily perform mixed queries. For example, get all accidents in ward `0` (or any other geometry) with at least three vehicles involved:

In [ ]:
gdf[gdf.within(london_areas.geometry.iloc[0]) & (gdf.Number_of_Vehicles==3)]

Or we can plot some query and color by some specific column (note how default Viridis colormap is used):

In [ ]:
gdf[gdf.within(london_areas.geometry.iloc[0]) & (gdf.Accident_Severity<=2)].plot(column="Number_of_Casualties")

# Common operations

Various geospatial operations are very common when doing analysis on geographical data. For example, we may want to calculate number of accidents per ward in 2005.

To do this, we perform **spatial join** between accidents dataset and London areas dataset.

**Note:** join will fail, if we have missing coordinates, so let's remove those from the dataframe:

In [ ]:
gdf = gdf[df.Location_Easting_OSGR.notnull() & df.Location_Northing_OSGR.notnull()]

In [ ]:
gdf.head()

Spatial join is performed by `gpd.sjoin`:

In [ ]:
gpd_london = gpd.sjoin(gdf[gdf.Year==2005], london_areas)

Note, that join can be performed with different types of geometric comparison and the default is `intersects` (sometimes you may need to go with `within`, but it's irrelevant in thi case as we have points in accidents dataset):

In [ ]:
gpd.sjoin?

We now know, in which city area each accident happened. Note, that the resulting dataframe only contains those accidents, which happened in London (see dataframe shape):

In [ ]:
gpd_london

One final touch:

In [ ]:
gpd_london.rename({"index_right":"city_area"}, axis=1, inplace=True)

We can now perform generic analysis:

In [ ]:
plt.figure(figsize=(6,6))
gpd_london.groupby("city_area").size().plot(kind="hist", range=(0, 1000), bins=10)
plt.xlabel("Number of accidents per city area");

In [ ]:
plt.figure(figsize=(6,6))
(1e6*gpd_london.groupby("city_area").size()/london_areas.area).plot(kind="hist", range=(0, 500), bins=10)
plt.xlabel("Number of accidents per city area/per km2");

However, it's interesting to look at the data we've got so far on a plot:

In [ ]:
london_areas = london_areas.join(gpd_london.groupby("city_area").size().rename("accidents_2005"))

In [ ]:
plt.figure(figsize=(6,6))
london_areas.plot(column="accidents_2005", ax=plt.gca())

sns.despine(left=True, bottom=True)
plt.gca().set_facecolor("white")

With this basic knowledge, you may dig deeper into the topic and explore relations with, for example, some demographic information (you'll need to search for it!) or other information. Try to figure out how to plot this on a map (you may find [Basemap Matplotlib toolkit](https://matplotlib.org/basemap/index.html) useful).